In [1]:
import sidechainnet as scn
from glob import glob
import os
import tqdm.notebook as tqdm


data_dir = "/net/pulsar/home/koes/jok120/scnmin221001/unmin"
%cd ~/

/net/pulsar/home/koes/jok120


In [2]:
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob(''.join(map(either, pattern)))

In [12]:
def get_corresponding_pkl(starting_pdb, dir_with_pkls):
    dirs, name = os.path.split(starting_pdb)
    pdbcode_chain, extension = os.path.splitext(name)
    pdbcode, chain = pdbcode_chain.split("_")
    if "#" in pdbcode:
        pdbcode = pdbcode.split("#")[1]
    new_basename = f"*{pdbcode.upper()}_*_{chain}.pkl"
    new_path_glob = os.path.join(dir_with_pkls, new_basename)
    try: 
        new_path = glob(new_path_glob)[0]
    except IndexError:      
        try:
            new_path = insensitive_glob(new_path_glob)[0]
        except IndexError:
            print(f"Could not find {new_basename} in {dir_with_pkls}")
            return None
    
    return new_path



In [13]:
len(glob(f"/net/pulsar/home/koes/jok120/openfold/data/train_structs/scnmin_structs0530/*.pdb"))

32994

In [15]:
failed = []
for protein in tqdm.tqdm(insensitive_glob(f"openfold/data/train_structs/scnmin_structs0530/*.pdb")):
    # print(protein)
    corresponding_protein_pkl = get_corresponding_pkl(protein, data_dir)
    if corresponding_protein_pkl is None:
        print(f"Could not find {protein}.")
        failed.append(protein)
        continue
    # print(corresponding_protein_pkl)
    p = scn.SCNProtein.from_pkl(corresponding_protein_pkl)
    # print(p)
    outpath = "openfold/data/train_structs/scnunmin_structs0530/" + os.path.basename(protein)
    # print(outpath)
    p.to_pdb(outpath)

  0%|          | 0/32994 [00:00<?, ?it/s]

In [ ]:



# Write this loop in a parallelized way
# Path: jk_research/notebooks/230612_make_scnunmin.ipynb
import sidechainnet as scn
from glob import glob
import os
# import tqdm.notebook as tqdm
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing



data_dir = "scnmin221001/unmin"
%cd ~/

def get_corresponding_pkl(starting_pdb, dir_with_pkls):
    dirs, name = os.path.split(starting_pdb)
    pdbcode_chain, extension = os.path.splitext(name)
    pdbcode, chain = pdbcode_chain.split("_")
    new_basename = f"{pdbcode.upper()}_*_{chain.upper()}.pkl"
    new_path_glob = os.path.join(dir_with_pkls, new_basename)
    try:
        new_path = insensitive_glob(new_path_glob)[0]
    except IndexError:
        print(f"Could not find {new_basename} in {dir_with_pkls}")
        return None
    return new_path


def make_unmin(protein):
    # print(protein)
    corresponding_protein_pkl = get_corresponding_pkl(protein, "/net/pulsar/home/koes/jok120/scnmin221001/unmin")
    # print(corresponding_protein_pkl)
    if corresponding_protein_pkl is None:
        print(f"Could not find corresponding pkl for {protein}")
        return protein
    p = scn.SCNProtein.from_pkl(corresponding_protein_pkl)
    # print(p)
    outpath = "/net/pulsar/home/koes/jok120/openfold/data/train_structs/scnunmin_structs0530/" + os.path.basename(protein)
    # print(outpath)
    p.to_pdb(outpath)
    return "okay"

with multiprocessing.Pool(60) as pool:
    failed = pool.map(make_unmin, glob(f"/net/pulsar/home/koes/jok120/openfold/data/train_structs/scnmin_structs0530/*.pdb"))


# Parallel(n_jobs=12)(delayed(make_unmin)(protein) for protein in tqdm(glob(f"openfold/data/train_structs/scnmin_structs0530/*.pdb"), total=32994))

In [ ]:
glob(f"/net/pulsar/home/koes/jok120/openfold/data/train_structs/scnmin_structs0530/*.pdb")

In [ ]:
for protein_pkl in glob(f"{data_dir}/*.pkl"):
    p = scn.SCNProtein.from_pkl(protein_pkl)
    print(p)
    break

In [ ]:
def get_scn_full_pattern_from_codechain_pattern(openfold_path):
    dirs, name = os.path.split(openfold_path)
    pdbcode_chain, extension = os.path.splitext(name)
    pdbcode, chain = pdbcode_chain.split("_")
    # scn_pattern = os.path.join(dirs, f"{pdbcode.upper()}_*_{chain.upper()}.pkl")
    scn_pattern = f"{pdbcode.upper()}_*_{chain.upper()}.pkl"
    return scn_pattern

In [ ]:
!find /net/pulsar/home/koes/jok120/openfold/data/train_structs/scnmin_structs0530/ -name "*.pdb" -type f 

In [ ]:
starting_files = 

In [ ]:
for scnmin_protein_fn in !ls ~/openfold/data/train_structs/scnmin_structs0530/*pdb:
    scn_style_name_pattern = get_scn_full_pattern_from_codechain_pattern(scnmin_protein_fn)
    print(scn_style_name_pattern)
    actual_file_unmin = next(glob(f"{data_dir}/{scn_style_name_pattern}"))
    p = scn.SCNProtein.from_pkl(actual_file_unmin)
    print(p)
    break

In [ ]:
glob(f"~/openfold/data/train_structs/scnmin_structs0530/*pdb", recursive=True)